<a href="https://colab.research.google.com/github/CienciaDatosUdea/002_EstudiantesAprendizajeEstadistico/blob/main/semestre2024-2/Sesiones/Sesion_11_SVM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Máquinas de Soporte Vectorial SVM

Sección 2.4.3 en https://arxiv.org/pdf/2204.04198

Este es un modelo con origenes que se remontan a los 60's y que se creó desde un punto de vista geométrico. Con este se puede llegar por un camino diferente a la función de coste y regularización a los que llegamos por el método Maximum Likelihood Estimator.

Empecemos con un problema de clasificación de unos datos cuyo i-esimos record $X^{(i)}$ tiene dos variables y que queremos predecir si pertenecen a una clase +1 y 0. El modelo de clasificación se condensa en tres parámetros $\theta_0,\theta_1,\theta_2$ que definen una linea (un hiperplano cuando pensemos en más dimensiones) a través de la ecuación $\theta^T X + \theta_0 = 0$ con $ \theta=(\theta_1,\theta_2)$. Los puntos de la linea son los puntos perpendiculares a $ \theta$. Los puntos del plano quedan clasificados por la región respecto por el signo que resulta del producto escalar $\theta^T X$. Si este producto es positivo están en la región que definimos como de clase +1 y si es negativo en la región definida como clase 0. Con solo argumentos de geometría se demuestra que encontrar los mejores parámetros para un conjunto de datos equivale a minimizar una función de coste parecida a las encontradas anteriormente.

La cantidad principal es la menor distancia que hay entre un punto cualquiera del plano (x1,x2) y el plano del modelo $\theta^T X + \theta_0 = 0$. Esta distancia está dada por $$ d(x, \theta) = \frac{\theta^T x + \theta_0}{| \theta |} $$
El problema de encontrar un buen hiperplano de clasificación consiste entonces en encontrar un conjunto de parámetros que la distancia al hiperplano sea tan grande como pósible o al menos mas grande que una región que escribiremos como $M | \theta |$ es decir resolver que para todo i se cumpla:
$$ y_i (\theta^T x + \theta_0) \ge  M | \theta | $$

Ahora, para simplificar, reescalamos $\theta$ con $ | \theta | = {1}{M}$ tal que la el problema a solucionar se convierte en $$ y_i (\theta^T x + \theta_0) \ge  1 $$

Este reescalamiento hace que buscar que M sea tan grande como posible se convierte en el problema de buscar $| \theta |$ sea tan pequeño como posible. El problema de encontrar el mejor modelo de clasificiación es ahora entonces minimizar $| \theta |$ con la condición que $ y_i (\theta^T x + \theta_0) \ge  1 $ para todo i. Esto se logra por el método de Lagrange. Buscamos minimizar:

$$ L = \frac{1}{2} | \theta |^2 - \sum_i [ \alpha_i  y_i (\theta^T x + \theta_0) -  1 ] $$

Observermos que los terminos se parecen a los que llegamos por MLE. El primero disminuye la complejidad y el segundo término depende del error que se comete en la clasificación. En una libreria como sklearn la siguiente función de coste se llamaría con un parámetro $C$ que multiplicaría el segundo termino. Con esto podemos graduar que tanta importancia se le da al segundo término sobre el primero:

Hay mas detalles técnicos a tener en cuenta que si lo necesitan pueden consultar en estas notas por ejemplo:
https://see.stanford.edu/materials/aimlcs229/cs229-notes3.pdf




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn.datasets import make_blobs
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Libraries for draw contours
def make_meshgrid(x, y, h=0.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


def plot_contoursExact(ax, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


In [ ]:
# Dataset Toys References
# https://scikit-learn.org/stable/datasets/toy_dataset.html
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html
# Dataset sinteticos
X0, y0 = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, 
    n_clusters_per_class=1
)
X1, y1 = make_moons(n_samples=100, noise=0.15, shuffle=True,  random_state=1)
X2, y2 = make_circles(n_samples=100, noise=0.05, shuffle=True,  random_state=1)
X3, y3 = make_blobs(n_samples=500, centers=3, n_features=2,shuffle=True, 
                    random_state=10)

fig, axs = plt.subplots(2,2)

axs[0, 0].plot(X0[:,0][y0==0],X0[:,1][y0==0],"ro", alpha=0.5)
axs[0, 0].plot(X0[:,0][y0==1],X0[:,1][y0==1],"bo", alpha=0.5)

# Dataset a moons
axs[0, 1].plot(X1[:,0][y1==0],X1[:,1][y1==0],"ro", alpha=0.5)
axs[0, 1].plot(X1[:,0][y1==1],X1[:,1][y1==1],"bo", alpha=0.5)

# Dataset circles
axs[1, 0].plot(X2[:,0][y2==0],X2[:,1][y2==0],"ro", alpha=0.5)
axs[1, 0].plot(X2[:,0][y2==1],X2[:,1][y2==1],"bo", alpha=0.5)

# Dataset circles
axs[1, 1].plot(X3[:,0][y3==0],X3[:,1][y3==0],"ro", alpha=0.5)
axs[1, 1].plot(X3[:,0][y3==1],X3[:,1][y3==1],"bo", alpha=0.5)
axs[1, 1].plot(X3[:,0][y3==2],X3[:,1][y3==2],"go", alpha=0.5)



In [ ]:
# Based on :
# https://rramosp.github.io/ai4eng.v1.20211.udea/content/NOTES%2003.03%20-%20SVM%20AND%20FEATURE%20TRANSFORMATION.html

In [ ]:
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1, shuffle=True)
#X, y = make_circles(n_samples=200, noise=0.1, shuffle=True,  random_state=1)

plt.plot(X[:,0][y==0],X[:,1][y==0],"ro", alpha=0.5)
plt.plot(X[:,0][y==1],X[:,1][y==1],"bo", alpha=0.5)

In [ ]:
clf = SVC(kernel= 'linear', C=1.0) # C denota la fuerza del segundo termino. C muy pequeño hace que minimizar theta es mas importante
# se vuelven cero, o una constante y a partir de cierto valor ya el segundo termino empiza a tener mas peso 
clf.fit(X, y)

#Countour plot
fig, ax = plt.subplots()
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
plt.plot(X[y==0][:,0],X[y==0][:,1],"bo", alpha=1)
plt.plot(X[y==1][:,0],X[y==1][:,1],"ro", alpha=1)
print(f"Training error:{clf.score(X, y):.3f}")


# Métodos de Kernel

Los métodos de Kernel transforman los datos en un espacio en el cual ellos tienen posibilidad de ser separados. Por ejemplo los circulos no hay manera que una frontera lineal los separe. Mas concretamente se trata de hacer una transformación $x \to \phi(x)$. Se determina una nueva función de coste que en vez de depender de $x_i$ va a depender de $\phi(x_i)$. El nombre de método de kernel viene del hecho del reemplazo
$$  \ket x_i, x_j \bra \to \ket \phi(x_i), \phi(x_j) = K(x_i, x_j) $$
donde se denota el kernel. Vamos a elegir diferentes kernels para los problemas anteriores. 

In [ ]:
X, y = make_circles(n_samples=200, noise=0.1, shuffle=True,  random_state=1)
clf = SVC(kernel= 'rbf', gamma=1) # El parametro gamma es un parametro relacionado con la complejidad del kernel
clf.fit(X, y)

#Countour plot
fig, ax = plt.subplots()
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
plt.plot(X[y==0][:,0],X[y==0][:,1],"bo", alpha=1)
plt.plot(X[y==1][:,0],X[y==1][:,1],"ro", alpha=1)
print(f"Training error:{clf.score(X, y):.3f}")


# Grid Search of parameter in SVM

Grid-search is used to find the optimal hyperparameters of a model which results in the most ‘accurate’ predictions.
https://towardsdatascience.com/grid-search-for-model-tuning-3319b259367e

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV


X, y = make_circles(n_samples=200, noise=0.1, shuffle=True,  random_state=1)

plt.plot(X[:,0][y==0],X[:,1][y==0],"ro", alpha=0.5)
plt.plot(X[:,0][y==1],X[:,1][y==1],"bo", alpha=0.5)

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

clf = GridSearchCV(estimator=SVC(),
             param_grid = parameters)

clf.fit(X, y)
sorted(clf.cv_results_.keys())

In [ ]:
clf.cv_results_

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
#Countour plot
fig, ax = plt.subplots()
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
plt.plot(X[y==0][:,0],X[y==0][:,1],"bo", alpha=1)
plt.plot(X[y==1][:,0],X[y==1][:,1],"ro", alpha=1)
print(f"Training error:{clf.score(X, y):.3f}")


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import  StandardScaler

from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#Data
X, y = make_circles(n_samples=1000, noise=0.1, shuffle=True,  random_state=1)

#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1080)

#Scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test) 


In [ ]:
#Grid search
parameters = {'kernel':('linear', 'rbf','poly'), 'C':[0.01, 0.1, 1, 10, 100, 1000]}


classifier = GridSearchCV(estimator = SVC(),
                          param_grid = parameters,
                          scoring='accuracy')
classifier.fit(X_train_scaled, y_train)

#Grid search plots and print
mean_scores = classifier.cv_results_['mean_test_score']
std_scores = classifier.cv_results_['std_test_score']
kernel = classifier.cv_results_['param_kernel']
c = classifier.cv_results_['param_C'].data

scores = {'C': c, 'kernel':kernel, 'mean_test_score': mean_scores, 'std_test_score': std_scores}
scores_df = pd.DataFrame(data = scores)

ax = sns.relplot(data=scores_df,
                 x='C',
                 y='mean_test_score',
                 marker='X',
                 hue='kernel',
                 height=5,
                 aspect=2)

ax.set(title='Grid search acccuracy', xlabel='C: Inverse of regularization strength', ylabel='Model accuracy', xscale = 'log')

print('Best estimator: '+ str(classifier.best_estimator_))
print('Parameters: ' + str(classifier.best_params_))
print('Accuracy: ' + str(classifier.best_score_))

In [ ]:
# Model 
model = SVC(kernel='rbf', C=1)
model.fit(X_train_scaled,y_train)
# Confusion matrix
y_predict = model.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_predict))
cm = metrics.confusion_matrix(y_test, y_predict)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.grid(False)
plt.show()

Tarea 11.1 
1. Implementar un SVM para clasificar los siguientes datasets: make_moons, make_circles y make_bloobs, para ello se deberá crear un grid search. 
2. Con los mejores párametros dibujar  las fronteras de clasificación
3. Con los mejores parámetros dibujar la matriz de confusion 

In [ ]:
#Nota: si quiere hacer grid search con diferentes tipos de kernel se puede pedir secuencial la búsqueda por ejemplo
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
#luego no podriamos graficar para comparar diferentes modelos pero podriamos pedir el mejor estimador de todos los evaluados